# Binary Performance Evaluator – Demo Completo

Neste exemplo vamos treinar um classificador simples e explorar várias funcionalidades do `BinaryPerformanceEvaluator`. Usaremos dados sintéticos.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from riskpilot.evaluation import BinaryPerformanceEvaluator
from riskpilot.synthetic import SyntheticVintageGenerator

In [ ]:
X, y = make_classification(n_samples=600, n_features=5, random_state=0)
df = pd.DataFrame(X, columns=[f'f{i}' for i in range(5)])
df['target'] = y
df['id'] = range(len(df))
df['date'] = pd.date_range('2024-01-01', periods=len(df), freq='D')
df['group'] = np.random.choice(['A','B'], size=len(df))

In [ ]:
train = df.iloc[:400].reset_index(drop=True)
test = df.iloc[400:].reset_index(drop=True)
model = LogisticRegression().fit(train[[f'f{i}' for i in range(5)]], train['target'])

In [ ]:
evaluator = BinaryPerformanceEvaluator(model=model, df_train=train, df_test=test, target_col='target', id_cols=['id'], date_col='date', group_col='group')

In [ ]:
metrics = evaluator.compute_metrics()
metrics

In [ ]:
evaluator.plot_confusion(evaluator.df_test['target'], evaluator.df_test['y_pred_proba'], threshold='ks')

In [ ]:
evaluator.plot_calibration()

In [ ]:
evaluator.plot_event_rate()

In [ ]:
evaluator.plot_psi()

In [ ]:
evaluator.plot_ks()

In [ ]:
evaluator.plot_group_radar()

In [ ]:
evaluator.plot_decile_ks()

### Teste de Stress

In [ ]:
gen = SyntheticVintageGenerator(id_cols=['id'], date_cols=['date']).fit(train)
evaluator.synthetic_gen = gen
stress = evaluator.run_stress_test()
stress['metrics']